In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
debug = 2
import numpy as np
import pandas
import time
import geopandas
import os
from pprint import pprint
pandas.set_option('display.max_colwidth', -1)
g = 0
print('ok')

ok


# Process data for one year

## Read variable names and descriptions

In [2]:
s = time.time()
thisyear = 2018
# 2018: Wrote out 73,056 tracts in 36 minutes 2 seconds!
# 2017: Wrote out 73,056 tracts in 35 minutes 27 seconds!
# 2016: Wrote out 73,056 tracts in 35 minutes 56 seconds!
# 2015: Wrote out 73,056 tracts in 33 minutes 34 seconds!
# 2014: Wrote out 73,056 tracts in 33 minutes 10 seconds!
# 2013: Wrote out 73,056 tracts in 34 minutes 23 seconds!
# 2012: Wrote out 73,056 tracts in 36 minutes 32 seconds!
# 2011: Wrote out 73,056 tracts in 32 minutes 45 seconds!
# 2010: Wrote out 73,057 tracts in 34 minutes 10 seconds!

category = 'gerrymandering'

basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'
yeardir = basedir + str(thisyear) + '/'

if (thisyear == 2016):
    rawdatadir = yeardir + 'rawdata/data/tab4/sumfile/prod/2012thru2016/group2/'
elif (thisyear in [2015, 2013, 2011]):
    rawdatadir = yeardir + 'rawdata/group2/'
elif (thisyear == 2014):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2010thru2014/group2/'
elif (thisyear == 2012):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2008thru2012/group2/'
else:
    rawdatadir = yeardir + 'rawdata/'
metadir = yeardir + 'metadata/'
vardir = yeardir + 'variables/'
geodir = yeardir + 'geography/'
estimates_dir = yeardir + 'estimates/'
margins_of_error_dir = yeardir + 'margins_of_error/'

extras_dir = '/home/idies/workspace/Storage/raddick/census/extras/'

all_variables_df = pandas.read_csv(vardir+'variables_acs_5yr_all.csv', low_memory=False, index_col='rownumber')

# pre-2017, description is blank for FILEID, FILETYPE, STUSAB, CHARITER, SEQUENCE, LOGRECNO - so fill in with blank
all_variables_df = all_variables_df.fillna('')

print('Done')

Done


## Figure out which variables and sequences we need for our work

See the code at the bottom of this file for how I got all the variables we are getting.

In [3]:
# all_variables_df[all_variables_df['description'].apply(lambda x: 'vacant' in x.lower())][0:10]

# # for i in range(11,50):
# #     print('\'B01001_{0:03d}\','.format(i))

# all_variables_df[all_variables_df['variable'].isin(['B25002_001','B25002_002','B25002_003'])]

In [4]:
# Which variables are we getting?
varlist = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','B01001_001']
# for_cra_analysis_mac_varlist = ['FILEID',
#  'FILETYPE',
#  'STUSAB',
#  'CHARITER',
#  'SEQUENCE',
#  'LOGRECNO',
# # 'B00001_001',
# # 'B00002_001',
#  'B01001_001',
#  'B02001_001',
#  'B02001_002',
#  'B02001_003',
#  'B08013_001',
#  'B11001_001',
#  'B11001_002',
#  'B11001_006',
#  'B11001_007',
#  'B11001A_001', 
#  'B11001B_001',
# 'B01001_011',
# 'B01001_012',
# 'B01001_013',
# 'B01001_014',
# 'B01001_015',
# 'B01001_016',
# 'B01001_017',
# 'B01001_018',
# 'B01001_019',
# 'B01001_020',
# 'B01001_021',
# 'B01001_022',
# 'B01001_023',
# 'B01001_024',
# 'B01001_025',
# 'B01001_026',
# 'B01001_027',
# 'B01001_028',
# 'B01001_029',
# 'B01001_030',
# 'B01001_031',
# 'B01001_032',
# 'B01001_033',
# 'B01001_034',
# 'B01001_035',
# 'B01001_036',
# 'B01001_037',
# 'B01001_038',
# 'B01001_039',
# 'B01001_040',
# 'B01001_041',
# 'B01001_042',
# 'B01001_043',
# 'B01001_044',
# 'B01001_045',
# 'B01001_046',
# 'B01001_047',
# 'B01001_048',
# 'B01001_049',                                                                
#  'B15003_001',
#  'B15003_002',
#  'B15003_003',
#  'B15003_004',
#  'B15003_005',
#  'B15003_006',
#  'B15003_007',
#  'B15003_008',
#  'B15003_009',
#  'B15003_010',
#  'B15003_011',
#  'B15003_012',
#  'B15003_013',
#  'B15003_014',
#  'B15003_015',
#  'B15003_016',
#  'B15003_017',
#  'B15003_018',
#  'B15003_019',
#  'B15003_020',
#  'B15003_021',
#  'B15003_022',
#  'B15003_023',
#  'B15003_024',
#  'B15003_025',
#  'B17001_001',
#  'B17001_002',
#  'B17001A_001',
#  'B17001A_002',
#  'B17001B_001',
#  'B17001B_002',
#  'B19013_001',
#  'B19013A_001',
#  'B19013B_001',
#  'B19113_001',
#  'B23025_002',
#  'B23025_005',
#  'B25002_001',
#  'B25002_002',
#  'B25002_003',
#  'B25003_001',
#  'B25003_002',
#  'B25003_003',
#  'B25003A_001',
#  'B25003A_002',
#  'B25003A_003',
#  'B25003B_001',
#  'B25003B_002',
#  'B25003B_003',
#  'B25077_001',
#  'B25034_001',
#  'B25035_001']

# all_variables_df[all_variables_df['variable'].isin(for_cra_analysis_mac_varlist)]['description'][5:50]

# Which sequences will we need?
sequence_list = all_variables_df[all_variables_df['variable'].isin(varlist)]['sequence_number'].drop_duplicates().tolist()
e = time.time()
g = g + (e-s)

sequence_list


#for_cra_analysis_mac_varlist



[1, 2]

In [5]:
s = time.time()

state_codes_df = pandas.read_csv(extras_dir+'statecodes.csv')
state_codes_df = state_codes_df.set_index('STUSAB')

estimates_df = pandas.DataFrame()
margins_of_error_df = pandas.DataFrame()
states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

for i in sequence_list:
    if (debug >= 1):
        print('Importing estimates and margins of error for sequence {0:,.0f}...'.format(i))
    #print(all_variables_df[all_variables_df['sequence_number'] == i]['variable'].tolist()[0:10])
    
    this_seq_estimates_df = pandas.DataFrame()
    this_seq_margins_of_error_df = pandas.DataFrame()
    for onestate in states:
#        if (debug >= 2):
#            print('\tImporting estimates and margins of error for {0:}...'.format(onestate))
        state_estimates_filename = rawdatadir + 'e{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)
        state_margins_of_error_filename = rawdatadir + 'm{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)
#         if ('e{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i) not in os.listdir(rawdatadir)):
#             print('{0:} not found!'.format(state_estimates_filename))

        onestate_estiamtes_df = pandas.read_csv(state_estimates_filename, header=None, sep=',', encoding='utf-8', low_memory=False)
        this_seq_estimates_df = this_seq_estimates_df.append(onestate_estiamtes_df)
        onestate_margins_of_error_df = pandas.read_csv(state_margins_of_error_filename, header=None, sep=',', encoding='utf-8', low_memory=False)
        this_seq_margins_of_error_df = this_seq_margins_of_error_df.append(onestate_margins_of_error_df)
    if (i >= 2):
        #print(this_seq_estimates_df.columns.tolist())
        this_seq_estimates_df = this_seq_estimates_df.drop([0,1,2,3,4,5], axis=1)  # ['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO']
        this_seq_margins_of_error_df = this_seq_margins_of_error_df.drop([0,1,2,3,4,5], axis=1)  # ['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO']

    this_seq_estimates_df.columns = all_variables_df[all_variables_df['sequence_number'] == i]['variable'].tolist()
    this_seq_margins_of_error_df.columns = all_variables_df[all_variables_df['sequence_number'] == i]['variable'].tolist()
    
    estimates_df = pandas.concat((estimates_df,this_seq_estimates_df), axis=1, sort=False)
    margins_of_error_df = pandas.concat((margins_of_error_df,this_seq_margins_of_error_df), axis=1, sort=False)
    
print('checking whether the dataset has the variables we need...')

#requested_variables = all_variables_df[all_variables_df['sequence_number'] == 1]['variable'].tolist()
requested_variables = varlist

variables_to_get = []
variables_not_found = []
for x in requested_variables:
    if x in estimates_df.columns:
        variables_to_get.append(x)
    else:
        variables_not_found.append(x)

        
# if (len(variables_not_found) > 0):
#     print('Variables not found:')
#     for y in variables_not_found:
#         print('{0:}: {1:}\n'.format(y, all_variables_df[all_variables_df['variable'] == y]['description'].values[0]))

print('keeping only the variables we need...')
estimates_df = estimates_df[variables_to_get]
margins_of_error_df = margins_of_error_df[variables_to_get]

#all_variables_df[all_variables_df['variable'].isin(variables_not_found)]['description']


e = time.time()
g = g + (e-s)

print('\nFound estimates for {0:,.0f} tracts and margins of error for {1:,.0f} tracts in {2:,.0f} minutes {3:,.0f} seconds!'.format(len(estimates_df), len(margins_of_error_df), np.floor((e-s)/60), np.floor((e-s)%60)))


#estimates_df['STUSAB']

Importing estimates and margins of error for sequence 1...
Importing estimates and margins of error for sequence 2...
checking whether the dataset has the variables we need...
keeping only the variables we need...

Found estimates for 290,795 tracts and margins of error for 290,795 tracts in 1 minutes 28 seconds!


## Read geographies

In [6]:
s = time.time()
geo_df = pandas.DataFrame()

states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

if (debug >= 1):
    print('Reading geography files...')
for onestate in states:
    if (thisyear >= 2016):
        filename = geodir+'{0:}.xlsx'.format(onestate)
    else:
        filename = geodir+'{0:}.xls'.format(onestate)
    if (debug == 2):
        print('Reading geography for {0:}...'.format(onestate))
    this_geo_df = pandas.read_excel(filename)
    geo_df = pandas.concat((geo_df, this_geo_df), sort=False)
    if (thisyear >= 2017):
        geo_df = geo_df.assign(STUSAB = geo_df['State'].apply(lambda x: x.lower()))
    else:
        geo_df = geo_df.assign(STUSAB = geo_df['STATE'].apply(lambda x: x.lower()))

geo_df = geo_df.rename(columns={'Geography ID': 'GEOID'})

if (debug >= 1):
#    print('Retaining only tractlevel geographies...')
     print('Retaining only tract- and block-group-level geographies...')
#geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140'))]
geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140')) | (geo_df['GEOID'].apply(lambda x: x[0:3] == '150'))]

print('backing up...')
geo_df_bk = geo_df

##geo_df = geo_df.set_index('GEOID')  #We'll set GEOID as index colum AFTER the merge

e = time.time()
g = g + (e-s)
print('Done in {0:.0f} minutes {1:,.0f} seconds!'.format( np.floor((e-s)/60), (e-s) % 60))

#geo_df.sample(1)
#print('skipping...')
geo_df.head(1).T

Reading geography files...
Reading geography for al...
Reading geography for ak...
Reading geography for az...
Reading geography for ar...
Reading geography for ca...
Reading geography for co...
Reading geography for ct...
Reading geography for de...
Reading geography for dc...
Reading geography for fl...
Reading geography for ga...
Reading geography for hi...
Reading geography for id...
Reading geography for il...
Reading geography for in...
Reading geography for ia...
Reading geography for ks...
Reading geography for ky...
Reading geography for la...
Reading geography for me...
Reading geography for md...
Reading geography for ma...
Reading geography for mi...
Reading geography for mn...
Reading geography for ms...
Reading geography for mo...
Reading geography for mt...
Reading geography for ne...
Reading geography for nv...
Reading geography for nh...
Reading geography for nj...
Reading geography for nm...
Reading geography for ny...
Reading geography for nc...
Reading geography for

,1767
State,AL
Logical Record Number,1768
GEOID,14000US01001020100
Geography Name,"Census Tract 201, Autauga County, Alabama"
STUSAB,al


## Join geography to estimates and margins of error

In [7]:
s = time.time()
# print('getting from backup...')
# estimates_df = estimates_df_bk
# margins_of_error_df = margins_of_error_df_bk
# geo_df = geo_df_bk

#.sort_values('Logical Record Number')
#estimates_df[['STUSAB','LOGRECNO']].dtypes #object, int64
#geo_df[['STUSAB', 'Logical Record Number']].dtypes

if (thisyear >= 2017):
    estimates_df = estimates_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
    margins_of_error_df = margins_of_error_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
# elif (thisyear == 2016):
#     estimates_df = estimates_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
#     margins_of_error_df = margins_of_error_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'LOGRECNO'])
else:
    estimates_df = estimates_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'LOGRECNO'])
    margins_of_error_df = margins_of_error_df.merge(geo_df, left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'LOGRECNO'])

estimates_df = estimates_df.set_index('GEOID')  # set the GEOID after we add the shapefiles
margins_of_error_df = margins_of_error_df.set_index('GEOID')  # set the GEOID after we add the shapefiles


print('Total estimate of B01001_001: {0:,.0f}'.format(estimates_df['B01001_001'].sum()))
print('Margin of error in B01001_001: {0:,.0f}'.format(margins_of_error_df['B01001_001'].sum()))

print('\n')

print('Writing data...')

estimates_df.to_csv(estimates_dir + 'estimates_acs{0:.0f}_tract_bg_{1:}.csv'.format(thisyear, category), encoding='utf-8')
print('Estimates written to estimates_acs{0:.0f}_tract_bg_{1:}.csv '.format(thisyear, category))
      
margins_of_error_df.to_csv(margins_of_error_dir + 'margins_of_error_acs{0:.0f}_tract_{1:}.csv'.format(thisyear, category), encoding='utf-8')
print('Margins of error written to margin_of_error_acs{0:.0f}_tract_{1:}.csv'.format(thisyear, category))

e = time.time()
g = g + (e-s)
print('\nWrote out {0:,.0f} tracts in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(estimates_df), np.floor((g)/60), np.floor((g)%60)))

#margins_of_error_df

Total estimate of B01001_001: 645,806,060
Margin of error in B01001_001: 97,634,641


Writing data...
Estimates written to estimates_acs2018_tract_bg_gerrymandering.csv 
Margins of error written to margin_of_error_acs2018_tract_gerrymandering.csv

Wrote out 290,795 tracts in 3 minutes 45 seconds!


In [9]:
z = pandas.read_csv(estimates_dir+'estimates_acs{0:}_tract_bg_gerrymandering.csv'.format(thisyear), encoding='utf-8', low_memory=False, index_col='GEOID')
z

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,B01001_001,State,Logical Record Number,Geography Name
GEOID,,,,,,,,,,
14000US01001020100,ACSSF,201800000.0,al,0,1,1768,1923,AL,1768,"Census Tract 201, Autauga County, Alabama"
14000US01001020200,ACSSF,201800000.0,al,0,1,1769,2028,AL,1769,"Census Tract 202, Autauga County, Alabama"
14000US01001020300,ACSSF,201800000.0,al,0,1,1770,3476,AL,1770,"Census Tract 203, Autauga County, Alabama"
14000US01001020400,ACSSF,201800000.0,al,0,1,1771,3831,AL,1771,"Census Tract 204, Autauga County, Alabama"
14000US01001020500,ACSSF,201800000.0,al,0,1,1772,9883,AL,1772,"Census Tract 205, Autauga County, Alabama"
...,...,...,...,...,...,...,...,...,...,...
15000US560459511001,ACSSF,201800000.0,wy,0,1,944,1435,WY,944,"Block Group 1, Census Tract 9511, Weston County, Wyoming"
15000US560459511002,ACSSF,201800000.0,wy,0,1,945,1851,WY,945,"Block Group 2, Census Tract 9511, Weston County, Wyoming"
15000US560459513001,ACSSF,201800000.0,wy,0,1,946,1225,WY,946,"Block Group 1, Census Tract 9513, Weston County, Wyoming"
